# Problem Setup

As input, we are given a graph $G = (V_G, E_G)$, where each vertex is a geographic position $s_i \in S^2$, and each edge $(i, j)$ has an associated (Olivier-Ricci) curvature $\kappa_{i, j} \in (-2, 1)$ and an associated latency $t_{i, j} \in \mathbb{R}_{\ge 0}$.

Intuitively, we want to return a surface in $\mathbb{R}^3$ that is the graph of a function $f : S^2 \to \mathbb{R}_{> 0}$ whose geodesics $g_{i, j}$ between $s_i$ and $s_j$ (and their missing $\rho$-coordinates) have length $\phi_{i, j}$ that is in a linear relationship with the latency.

The strategy to realize this intuition is to create a mesh $M = (V_M, E_M)$ supported on a subset of $S^2$ that contains our input positions. Let $P$ be the support. Then for each $s_i \in P$, we want to assign a $\rho_i \in \mathbb{R}_{> 0}$, which in turn gives a point $v_i = (s_i, \rho_i) \in V$.

In [1]:
import itertools

import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
from scipy import linalg
from scipy import sparse
from scipy.sparse.linalg import eigsh, splu


In [2]:
class SphereMesh:
    '''
    Representation of a mesh that is "approximately" spherical. In particular,
    projecting the vertices of the mesh onto the unit sphere will yield a
    geodesic polyhedron with icosahedral symmetry.
    '''

    def __init__(self, frequency):
        self._directions, self._edges, self._faces, self._c \
            = self._initial_mesh(frequency)
        self._rho = np.ones(self._directions.shape[0])
        self._updates = 0

    def _initial_mesh(self, frequency):
        '''
        Compute a geodesic polyhedron with icosahedral symmetry.
        '''

        # Start by defining a simple icosahedron.
        p = (2 / (5 + 5**0.5))**0.5
        q = (2 / (5 - 5**0.5))**0.5
        icosahedron_vertices = np.array([
            [0, p, q], [0, p, -q], [0, -p, q], [0, -p, -q],
            [q, 0, p], [-q, 0, p], [q, 0, -p], [-q, 0, -p],
            [p, q, 0], [p, -q, 0], [-p, q, 0], [-p, -q, 0],
        ])

        # The following generates the edges
        #
        # for i in range(12):
        #     for j in range(i + 1, 12):
        #         vi = icosahedron_vertices[i,:]
        #         vj = icosahedron_vertices[j,:]
        #         if np.isclose(linalg.norm(vi - vj), 2 * p):
        #             print(f'({i}, {j}),')
        icosahedron_edges = [
            (0, 2), (0, 4), (0, 5), (0, 8), (0, 10),
            (1, 3), (1, 6), (1, 7), (1, 8), (1, 10),
            (2, 4), (2, 5), (2, 9), (2, 11), (3, 6),
            (3, 7), (3, 9), (3, 11), (4, 6), (4, 8),
            (4, 9), (5, 7), (5, 10), (5, 11), (6, 8),
            (6, 9), (7, 10), (7, 11), (8, 10), (9, 11),
        ]

        # The following generates the faces (oriented counterclockwise)
        #
        # for i in range(12):
        #     for j in range(i + 1, 12):
        #         for k in range(i + 1, 12):
        #             vi = icosahedron_vertices[i,:]
        #             vj = icosahedron_vertices[j,:]
        #             vk = icosahedron_vertices[k,:]
        #             v_sum = vi + vj + vk
        #             v_cross = np.cross(vj - vi, vk - vi)
        #             if (np.isclose(linalg.norm(vi - vj), 2 * p) and
        #                     np.isclose(linalg.norm(vj - vk), 2 * p) and
        #                     np.isclose(linalg.norm(vk - vi), 2 * p) and
        #                     v_sum @ v_cross > 0):
        #                 print(f'({i}, {j}, {k}),')
        icosahedron_faces = [
            (0, 2, 4), (0, 4, 8), (0, 5, 2), (0, 8, 10), (0, 10, 5),
            (1, 3, 7), (1, 6, 3), (1, 7, 10), (1, 8, 6), (1, 10, 8),
            (2, 5, 11), (2, 9, 4), (2, 11, 9), (3, 6, 9), (3, 9, 11),
            (3, 11, 7), (4, 6, 8), (4, 9, 6), (5, 7, 11), (5, 10, 7),
        ]

        # Now that we have defined the icosahedron, we can subdivide the faces
        # and then project them to the unit sphere.

        # The vertices are organized by
        #     [
        #         icosahedron vertices,
        #         vertices from the icosahedron edges,
        #         vertices from the icosahedron faces,
        #     ].
        vertices = np.zeros((10 * frequency**2 + 2, 3))

        # Edges are organized in an adjacency list fashion.
        edges = [[] for _ in range(vertices.shape[0])]

        # Faces are represented by (i, j, k) tuples, where i -> j -> k is
        # oriented counterclockwise.
        faces = []

        # Vertices from icosahedron vertices
        vertices[:12, :] = icosahedron_vertices
        index = 12

        # Vertices from icosahedron edges
        edge_index_offset = index
        for e in icosahedron_edges:
            v0 = icosahedron_vertices[e[0], :]
            v1 = icosahedron_vertices[e[1], :]
            d = v1 - v0

            for i in range(1, frequency):
                # Recall that interpolation between v0 and v1 can be written as
                # v = v0 + lambda * (v1 - v0) = v0 + lambda * d. Then we just
                # have to normalize v.
                # We use the name `lam` here since `lambda` is reserved.
                lam = i / frequency
                v = v0 + lam * d
                vertices[index, :] = v / linalg.norm(v)
                index += 1

        # Vertices from icosahedron faces. At the same time, keep track of the
        # edges and the faces.
        for f in icosahedron_faces:
            v0 = icosahedron_vertices[f[0], :]
            v1 = icosahedron_vertices[f[1], :]
            d1 = v1 - v0
            v2 = icosahedron_vertices[f[2], :]
            d2 = v2 - v0

            # Vertices
            face_vertices = [[] for _ in range(frequency + 1)]
            face_vertices[0].append(f[0])
            if f[1] > f[0]:
                edge_index = edge_index_offset \
                    + icosahedron_edges.index((f[0], f[1])) * (frequency - 1)
                for i in range(frequency - 1):
                    face_vertices[0].append(edge_index + i)
            else:
                edge_index = edge_index_offset \
                    + icosahedron_edges.index((f[1], f[0])) * (frequency - 1)
                for i in range(frequency - 1):
                    face_vertices[0].append(edge_index + frequency - 2 - i)
            face_vertices[0].append(f[1])
            if f[2] > f[0]:
                edge_index = edge_index_offset \
                    + icosahedron_edges.index((f[0], f[2])) * (frequency - 1)
                for i in range(frequency - 1):
                    face_vertices[i + 1].append(edge_index + i)
            else:
                edge_index = edge_index_offset \
                    + icosahedron_edges.index((f[2], f[0])) * (frequency - 1)
                for i in range(frequency - 1):
                    face_vertices[i + 1].append(edge_index + frequency - 2 - i)
            face_vertices[frequency].append(f[2])

            for i in range(1, frequency - 1):
                lam_i = i / frequency
                v_prime = v0 + lam_i * d2
                for j in range(1, frequency - i):
                    lam_j = j / frequency
                    v = v_prime + lam_j * d1
                    vertices[index, :] = v / linalg.norm(v)
                    face_vertices[i].append(index)
                    index += 1

            if f[2] > f[1]:
                edge_index = edge_index_offset \
                    + icosahedron_edges.index((f[1], f[2])) * (frequency - 1)
                for i in range(frequency - 1):
                    face_vertices[i + 1].append(edge_index + i)
            else:
                edge_index = edge_index_offset \
                    + icosahedron_edges.index((f[2], f[1])) * (frequency - 1)
                for i in range(frequency - 1):
                    face_vertices[i + 1].append(edge_index + frequency - 2 - i)

            # Edges and faces
            for i in range(frequency):
                for j in range(frequency - i):
                    s = face_vertices[i][j]
                    t = face_vertices[i][j + 1]
                    u = face_vertices[i + 1][j]
                    edges[s].append(t)
                    edges[t].append(u)
                    edges[u].append(s)
                    faces.append((s, t, u))
            for i in range(1, frequency):
                for j in range(frequency - i):
                    s = face_vertices[i][j]
                    t = face_vertices[i - 1][j + 1]
                    u = face_vertices[i][j + 1]
                    edges[s].append(t)
                    edges[t].append(u)
                    edges[u].append(s)
                    faces.append((s, t, u))

        c = {}
        for i, j, k in faces:
            c[i,j] = k
            c[j,k] = i
            c[k,i] = j

        return vertices, edges, faces, c

    def get_directions(self):
        '''
        Return the (normalized) directions of the mesh. This is useful for
        computing partial derivatives later on. The output of this function
        should be treated as "read only."
        '''
        return self._directions

    def get_vertices(self):
        '''
        Return the vertices of the mesh.
        '''

        return np.multiply(self._directions, np.reshape(self._rho, (-1, 1)))

    def get_edges(self):
        '''
        Return the edges of the mesh adjacency list format. The output of this
        function should be treated as "read only."
        '''
        return self._edges

    def get_faces(self):
        '''
        Return a list of triples (i, j, k) representing the (indices of the)
        faces of the mesh. Each i -> j -> k is oriented counterclockwise. The
        output of this function should be treated as "read only."
        '''

        return self._faces

    def get_c(self):
        '''
        Return a map from pairs of indices to indices where (i, j) maps to k
        precisely when i -> j -> k is a face oriented counterclockwise. The
        output of this function should be treated as "read only."
        '''

        return self._c

    def get_rho(self):
        '''
        Return the magnitudes of the vertices of this mesh, ordered in the same
        way as the output of `get_vertices`.
        '''

        return np.copy(self._rho)

    def set_rho(self, rho):
        '''
        Set the magnitudes of the vertices of this mesh, ordered in the same
        way as the output of `get_vertices`.
        '''
        self._rho = np.copy(rho)
        self._updates += 1

    def updates(self):
        '''
        Return the number of times `set_rho` has been called. This function is
        an easy (O(1)) way to determine whether the mesh has been updated.
        '''

        return self._updates

    def nearest_direction_index(self, direction):
        '''
        Find the index of the direction on the mesh closest to the input
        direction.
        '''

        return np.argmax(self._directions @ direction)

    @staticmethod
    def latitude_longitude_to_direction(latitude, longitude):
        '''
        Takes in latitude (in [-90, 90]) and longitude (in (-180, 180]) and
        returns the corresponding direction on the unit sphere in R^3. For
        efficiency, the ranges of the inputs are not checked.
        '''

        latitude = np.radians(latitude)
        longitude = np.radians(longitude)

        return np.array([
            np.cos(longitude) * np.cos(latitude),
            np.sin(longitude) * np.cos(latitude),
            np.sin(latitude)
        ])


In [3]:
class Animation3D:
    '''
    Animator for meshes. Frames should be added one-by-one via `add_frame`, and
    then the Plotly animation figure is returned via `get_fig`.
    '''

    def __init__(self):
        self.clear_frames()

    def clear_frames(self):
        '''
        Reset the animation.
        '''

        self._frames = []
        self._radius = 0

    def add_frame(self, mesh):
        '''
        Add a frame that contains `mesh`.
        '''

        vertices = mesh.get_vertices()
        faces = mesh.get_faces()
        data = ff.create_trisurf(
            vertices[:,0], vertices[:,1], vertices[:,2],
            faces, colormap='#1f77b4',
            show_colorbar=False, plot_edges=True,
            aspectratio=dict(x=1, y=1, z=1)
        ).data
        self._radius = max(self._radius, np.max(vertices))
        frame = go.Frame(data=data)
        frame['name'] = str(len(self._frames))
        self._frames.append(frame)

    def get_fig(self, duration=0):
        '''
        Return a Plotly figure that can be displayed by calling its `.show()`
        method. The `duration` parameter controls the speed of the animation
        (lower is faster; must be non-negative).
        '''

        fig_dict = {
            'data': self._frames[0].data,
            'layout': {},
            'frames': self._frames,
        }

        fig_dict['layout']['title'] = {
            'text': 'Sphere Optimization',
        }
        fig_dict['layout']['width'] = 600
        fig_dict['layout']['height'] = 600

        axis_format = {
            'color': '#ffffff',
            'range': [-self._radius, self._radius],
            'showaxeslabels': False,
            'showticklabels': False,
            'showbackground': False,
            'showspikes': False,
        }
        fig_dict['layout']['scene'] = {}
        fig_dict['layout']['scene']['xaxis'] = axis_format
        fig_dict['layout']['scene']['yaxis'] = axis_format
        fig_dict['layout']['scene']['zaxis'] = axis_format
        fig_dict['layout']['scene']['hovermode'] = False

        fig_dict['layout']['updatemenus'] = [
            {
                'type': 'buttons',
                'buttons': [
                    {
                        'label': 'Play',
                        'method': 'animate',
                        'args': [None, {'frame': {'duration': duration},
                                        'fromcurrent': True,
                                        'transition': {'duration': 0}}],
                    },
                    {
                        'label': 'Pause',
                        'method': 'animate',
                        'args': [[None], {'frame': {'duration': 0},
                                          'mode': 'immediate',
                                          'transition': {'duration': 0}}],
                    },
                ],
            }
        ]

        sliders_dict = {
            'currentvalue': {
                'prefix': 'batch: ',
                'xanchor': 'right',
            },
            'transition': {'duration': 0},
            'steps': [
                {
                    'args': [
                        [str(i)],
                        {
                            'mode': 'immediate',
                            'frame': {'duration': duration},
                            'transition': {'duration': 0},
                        },
                    ],
                    'label': str(i),
                    'method': 'animate',
                }
                for i, _ in enumerate(self._frames)
            ]
        }
        fig_dict['layout']['sliders'] = [sliders_dict]
        return go.Figure(fig_dict)


# Laplacian

## Laplacian Computation
Some notation first. If $i$ and $j$ are two indices vertices for which $(v_i, v_j)$ is an edge, let $c(i, j)$ be the index such that $v_i \to v_j \to v_{c(i, j)}$ traces a triangle counterclockwise. Note that this index exists and is unique assuming we have a mesh without boundary.

We have the following (standard) definition of the Laplace-Beltrami operator on a mesh: $$\begin{aligned}
    N_{i, j} &\triangleq (v_i - v_{c(i, j)}) \times (v_j - v_{c(i, j)}), & \text{Outward normal of triangle $v_i \to v_j \to v_{c(i, j)}$} \\
    A_{i, j} &\triangleq \frac{1}{2}\|N_{i, j}\|_2, & \text{Area of triangle $v_i \to v_j \to v_{c(i, j)}$} \\
    D_{i, j} &\triangleq \begin{cases}
        \displaystyle\frac{1}{3}\sum_{\substack{k \\ \text{$(v_i, v_k)$ is an edge}}}A_{i, k} & \text{if $i = j$}, \\
        0 & \text{otherwise},
    \end{cases} & \text{Vertex triangle areas; diagonal} \\
    \cot(\theta_{i, j}) &= \frac{(v_i - v_{c(i, j)}) \cdot (v_j - v_{c(i, j)})}{2A_{i, j}}, & \text{Cotangent of $\angle v_iv_{c(i, j)}v_j$} \\
    (L_C)_{i, j} &\triangleq \begin{cases}
        \displaystyle\frac{1}{2}(\cot(\theta_{i, j}) + \cot(\theta_{j, i})) & \text{if $(i, j)$ is an edge}, \\
        \displaystyle-\frac{1}{2}\sum_{\substack{k \\ \text{$(v_i, v_k)$ is an edge}}}(\cot(\theta_{i, k}) + \cot(\theta_{k, i})) & \text{if $i = j$}, \\
        0 & \text{otherwise},
    \end{cases} & \text{Cotangent operator; sparse} \\
    L &\triangleq D^{-1}L_C.
\end{aligned}$$

In [4]:
class LaplacianForward:
    '''
    Implementation of the Laplace-Beltrami operator on a mesh.
    '''

    def __init__(self, mesh):
        self._mesh = mesh
        self._updates = self._mesh.updates() - 1
        self._v = None
        self._e = self._mesh.get_edges()
        self._c = self._mesh.get_c()

        self._V = len(self._e)

        # A map (i, j) -> N_ij
        self.N = None

        # A map (i, j) -> A_ij
        self.A = None

        # A sparse matrix D
        self.D = None

        # A map (i, j) -> cot(theta_ij)
        self.cot = None

        # A sparse matrix L_C
        self.LC = None

        # A sparse matrix L
        self.L = None

    def _calc_N(self):
        v = self._v
        e = self._e
        c = self._c
        return {(i, j): np.cross(v[i] - v[c[i,j]], v[j] - v[c[i,j]])
                for i, es in enumerate(e)
                for j in es}

    def _calc_A(self):
        return {(i, j): linalg.norm(N) / 2
                for (i, j), N in self.N.items()}

    def _calc_D(self):
        e = self._e
        A = self.A
        return sparse.diags([sum(A[i,j] for j in e) / 3
                             for i, e in enumerate(e)])

    def _calc_cot(self):
        v = self._v
        e = self._e
        c = self._c
        A = self.A
        return {(i, j): (v[i] - v[c[i,j]]) @ (v[j] - v[c[i,j]]) / (2 * A[i,j])
                for i, es in enumerate(e)
                for j in es}

    def _calc_LC(self):
        row = []
        col = []
        data = []
        for (i, j), cot_ij in self.cot.items():
            half_cot_ij = cot_ij / 2

            row.append(i)
            col.append(j)
            data.append(half_cot_ij)

            row.append(j)
            col.append(i)
            data.append(half_cot_ij)

            row.append(i)
            col.append(i)
            data.append(-half_cot_ij)

            row.append(j)
            col.append(j)
            data.append(-half_cot_ij)
        return sparse.coo_array((data, (row, col)),
                                 shape=(self._V, self._V)).tocsc()

    def calc_L(self):
        if self._updates != self._mesh.updates():
            self._updates = self._mesh.updates()
            self._v = self._mesh.get_vertices()

            self.N = self._calc_N()
            self.A = self._calc_A()
            self.D = self._calc_D()
            self.D_inv = sparse.diags(1 / self.D.data.flatten())
            self.cot = self._calc_cot()
            self.LC = self._calc_LC()
            self.L = self.D_inv @ self.LC

        return self.L


## Laplacian Gradient Computation

For the ease of notation (and the avoidance of edge cases), assume that we are using the spherical setup, so $v_\ell = \rho_\ell s_\ell$.

We compute $$\begin{aligned}
    \frac{\partial v_i}{\partial \rho_\ell} &= \begin{cases}
        s_i & \text{if $\ell = i$}, \\
        0 & \text{otherwise},
    \end{cases} \\
    \frac{\partial N_{i, j}}{\partial \rho_\ell} &= \begin{cases}
        (v_{c(i, j)} - v_j) \times \frac{\partial v_\ell}{\partial \rho_\ell} & \text{if $\ell = i$}, \\
        (v_i - v_{c(i, j)}) \times \frac{\partial v_\ell}{\partial \rho_\ell} & \text{if $\ell = j$}, \\
        (v_j - v_i) \times \frac{\partial v_\ell}{\partial \rho_\ell} & \text{if $\ell = c(i, j)$}, \\
        0 & \text{otherwise},
    \end{cases} \\
    \frac{\partial A_{i, j}}{\partial \rho_\ell} &= \frac{1}{4A_{i, j}}N_{i, j} \cdot \frac{\partial N_{i, j}}{\partial \rho_\ell}, \\
    \left(\frac{\partial D}{\partial \rho_\ell}\right)_{i, j} &= \begin{cases}
        \displaystyle\frac{1}{3}\sum_{\substack{k \\ \text{$(v_i, v_k)$ is an edge}}}\frac{\partial A_{i, k}}{\partial \rho_\ell} & \text{if $i = j$}, \\
        0 & \text{otherwise},
    \end{cases} \\
    \frac{\partial}{\partial \rho_\ell}\cot(\theta_{i, j}) &= \begin{cases}
        \displaystyle\frac{(v_j - v_{c(i, j)}) \cdot \frac{\partial v_\ell}{\partial \rho_\ell} - 2\cot(\theta_{i, j})\frac{\partial A_{i, j}}{\partial \rho_\ell}}{2A_{i, j}} & \text{if $\ell = i$}, \\
        \displaystyle\frac{(v_i - v_{c(i, j)}) \cdot \frac{\partial v_\ell}{\partial \rho_\ell} - 2\cot(\theta_{i, j})\frac{\partial A_{i, j}}{\partial \rho_\ell}}{2A_{i, j}} & \text{if $\ell = j$}, \\
        \displaystyle\frac{(2v_{c(i, j)} - v_i - v_j) \cdot \frac{\partial v_\ell}{\partial \rho_\ell} - 2\cot(\theta_{i, j})\frac{\partial A_{i, j}}{\partial \rho_\ell}}{2A_{i, j}} & \text{if $\ell = c(i, j)$}, \\
        0 & \text{otherwise},
    \end{cases} \\
    \left(\frac{\partial L_C}{\partial \rho_\ell}\right)_{i, j} &= \begin{cases}
        \displaystyle\frac{1}{2}\left(\frac{\partial}{\partial \rho_\ell}\cot(\theta_{i, j}) + \frac{\partial}{\partial \rho_\ell}\cot(\theta_{j, i})\right) & \text{if $(i, j)$ is an edge}, \\
        \displaystyle-\frac{1}{2}\sum_{\substack{k \\ \text{$(v_i, v_k)$ is an edge}}}\left(\frac{\partial}{\partial \rho_\ell}\cot(\theta_{i, k}) + \frac{\partial}{\partial \rho_\ell}\cot(\theta_{k, i})\right) & \text{if $i = j$}, \\
        0 & \text{otherwise},
    \end{cases} \\
    \frac{\partial L}{\partial \rho_\ell} &= D^{-1}\left(\frac{\partial L_C}{\partial \rho_\ell} - \frac{\partial D}{\partial \rho_\ell}L\right).
\end{aligned}$$

In [5]:
class LaplacianReverse:
    '''
    Implementation of the gradient of the Laplace-Beltrami operator on a mesh.
    This implementation assumes the l-th partial affects only the l-th vertex.
    '''

    def __init__(self, mesh, laplacian_forward=None):
        self._mesh = mesh
        self._updates = self._mesh.updates() - 1
        self._v = None
        self._e = self._mesh.get_edges()
        self._c = self._mesh.get_c()

        self._V = len(self._e)

        self._dif_v = None
        self._ls = None

        self._laplacian_forward = laplacian_forward
        if self._laplacian_forward is None:
            self._laplacian_forward = LaplacianForward(mesh)

        self._N = None
        self._A = None
        self._D = None
        self._cot = None
        self._LC = None
        self._L = None

        # Derivatives are stored as maps sending l to the partial with respect
        # to rho_l. The types of the outputs of the maps match the types of
        # what are being differentiated.
        self.dif_N = None
        self.dif_A = None
        self.dif_D = None
        self.dif_cot = None
        self.dif_LC = None
        self.dif_L = None

    def _calc_dif_N(self, l):
        dif_N = {}
        v = self._v
        e = self._e
        c = self._c
        dif_v = self._dif_v[l]
        for i, es in enumerate(e):
            vi = v[i]
            for j in es:
                k = c[i,j]
                vj = v[j]
                vk = v[k]
                if l == i:
                    dif_N[i,j] = np.cross(vk - vj, dif_v)
                elif l == j:
                    dif_N[i,j] = np.cross(vi - vk, dif_v)
                elif l == k:
                    dif_N[i,j] = np.cross(vj - vi, dif_v)

                # For efficiency, only store the nonzero values
        return dif_N

    def _calc_dif_A(self, l):
        e = self._e
        N = self._N
        A = self._A
        dif_N = self.dif_N[l]
        return {(i, j): (N[i,j] @ dif_N[i,j]) / (4 * A[i,j])
                for i, es in enumerate(e)
                for j in es
                if (i, j) in dif_N}

    def _calc_dif_D(self, l):
        e = self._e
        dif_A = self.dif_A[l]
        return sparse.diags([sum(dif_A[i,j] for j in es if (i, j) in dif_A) / 3
                             for i, es in enumerate(e)])

    def _calc_dif_cot(self, l):
        dif_cot = {}
        v = self._v
        e = self._e
        c = self._c
        dif_v = self._dif_v[l]
        dif_A = self.dif_A[l]
        vl = v[l]
        for j in e[l]:
            k = c[l,j]
            vj = v[j]
            vk = v[k]
            dif_cot[l,j] = (((vj - vk) @ dif_v
                             - 2 * self._cot[l,j] * dif_A[l,j])
                            / (2 * self._A[l,j]))
            dif_cot[k,l] = (((vk - vj) @ dif_v
                             - 2 * self._cot[k,l] * dif_A[k,l])
                            / (2 * self._A[k,l]))
            dif_cot[j,k] = (((2 * vl - vj - vk) @ dif_v
                             - 2 * self._cot[j,k] * dif_A[j,k])
                            / (2 * self._A[j,k]))
        return dif_cot

    def _calc_dif_LC(self, l):
        dif_cot = self.dif_cot[l]
        row = []
        col = []
        data = []
        for (i, j), dif_cot_ij in dif_cot.items():
            half_dif_cot_ij = dif_cot_ij / 2

            row.append(i)
            col.append(j)
            data.append(half_dif_cot_ij)

            row.append(j)
            col.append(i)
            data.append(half_dif_cot_ij)

            row.append(i)
            col.append(i)
            data.append(-half_dif_cot_ij)

            row.append(j)
            col.append(j)
            data.append(-half_dif_cot_ij)
        return sparse.coo_array((data, (row, col)),
                                 shape=(self._V, self._V)).tocsc()

    def calc_dif_L(self, dif_v, ls=None):
        if self._ls is None:
            self._ls = range(self._V)
        if ls is None:
            ls = range(self._V)

        self._laplacian_forward.calc_L()
        self._N = self._laplacian_forward.N
        self._A = self._laplacian_forward.A
        self._D = self._laplacian_forward.D
        self._D_inv = self._laplacian_forward.D_inv
        self._cot = self._laplacian_forward.cot
        self._LC = self._laplacian_forward.LC
        self._L = self._laplacian_forward.L

        if self._updates != self._mesh.updates() or self._ls != ls:
            self._updates = self._mesh.updates()
            self._v = self._mesh.get_vertices()
            self._dif_v = dif_v
            self._ls = ls

            self.dif_N = {l: self._calc_dif_N(l) for l in self._ls}
            self.dif_A = {l: self._calc_dif_A(l) for l in self._ls}
            self.dif_D = {l: self._calc_dif_D(l) for l in self._ls}
            self.dif_cot = {l: self._calc_dif_cot(l) for l in self._ls}
            self.dif_LC = {l: self._calc_dif_LC(l) for l in self._ls}
            self.dif_L = {l: self._D_inv @
                             (self.dif_LC[l] - self.dif_D[l] @ self._L)
                          for l in self._ls}

        return self.dif_L


# Geodesic Distance via Heat Method

We follow the method found [here](https://www.cs.cmu.edu/~kmcrane/Projects/HeatMethod/) to compute the geodesic distance between points on our mesh.

## Geodesic Computation

Say we want to find the geodesic distances to a set of points $\gamma$. Following the strategy from [here](https://www.cs.cmu.edu/~kmcrane/Projects/HeatMethod/), we use the (approximate) heat flow $u^\gamma$, where $$\begin{aligned}
    t &\triangleq (\text{mean spacing between mesh points})^2, & \text{Adjustable parameter} \\
    \delta^\gamma &\triangleq \begin{cases}
        1 & \text{if $v_i \in \gamma$}, \\
        0 & \text{if $v_i \not\in \gamma$},
    \end{cases} & \text{Heat source} \\
    u^\gamma &\triangleq (D - tL_C)^{-1}\delta^\gamma & \text{Heat flow}
\end{aligned}$$

We can then compute $$\begin{aligned}
    q_{i, j} &\triangleq u^\gamma_i(v_{c(i, j)} - v_j), \\
    m_{i, j} &\triangleq q_{i, j} + q_{j, c(i, j)} + q_{c(i, j), i}, \\
    (\widetilde{\nabla} u^\gamma)_{i, j} &\triangleq N_{i, j} \times m_{i, j}, \\
    X^\gamma_{i, j} &\triangleq -\frac{(\widetilde{\nabla} u^\gamma)_{i, j}}{\|(\widetilde{\nabla} u^\gamma)_{i, j}\|_2}, \\
    p_{i, j} &\triangleq \cot(\theta_{i, j})(v_j - v_i), \\
    (\nabla \cdot X^\gamma)_i &= \frac{1}{2}\sum_{\substack{k \\ \text{$(v_i, v_k)$ is an edge}}}(p_{i, k} - p_{c(i, k), i}) \cdot X^\gamma_{i, k}, \\
    \phi^\gamma &= L_C^+ \cdot (\nabla \cdot X^\gamma).
\end{aligned}$$ Here, $L_C^+$ is the [pseudoinverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse) of $L_C$ (as it is singular). Note that the integrated divergence can be thought of as taking a sum over triangles $v_i \to v_k \to v_{c(i, k)}$.

Note that we're being careful about which pieces have a dependence on $\gamma$, as we can reuse certain computations if we want to compute distances from multiple sources. We can get the distance matrix (that is, get rid of the $\gamma$ dependence) from $$\phi_{i, j} = \left(\phi^{\{v_j\}}\right)_i.$$

In [6]:
class GeodesicForward:
    '''
    Implementation of the heat method for geodesic distance computation.
    '''

    def __init__(self, mesh, laplacian_forward=None):
        '''
        Quantities that can be naturally realized as matrices are stored as such
        (sometimes sparsely if reasonable). All other quantities have a one to
        one correspondence to pairs (v, f), where f is a face containing the
        vertex v. In other words, they are in correspondence to
        `mesh.get_all_faces()`, and are thus stored in the same format. If we
        think of the former of a map from vertices to lists of faces, then the
        latter is a map from vertices to lists of quantities. For efficiency
        reasons, these maps are stored as `list`s.
        '''

        self._mesh = mesh
        self._updates = self._mesh.updates() - 1
        self._v = None
        self._e = self._mesh.get_edges()
        self._c = self._mesh.get_c()

        self._V = len(self._e)

        self._laplacian_forward = laplacian_forward
        if self._laplacian_forward is None:
            self._laplacian_forward = LaplacianForward(mesh)

        self.N = None
        self.A = None
        self.D = None
        self.cot = None
        self.LC = None

        # A float
        self.h2 = None

        # An object with a `.solve` method that takes a vector as input and
        # returns a vector
        self.D_h2LC_inv = None

        # An object with a `.solve` method that takes a vector as input and
        # returns a vector
        self.LC_inv = None

        # An iterable containing indices
        self._gamma = None

        # A vector
        self.u = None

        # A vector
        self.q = None

        # A map (i, j) -> m_ij
        self.m = None

        # A map (i, j) -> u_ij
        self.grad_u = None

        # A map (i, j) -> X_ij
        self.X = None

        # A map (i, j) -> p_ij
        self.p = None

        # A matrix
        self.div_X = None

        # A matrix
        self.phi = None

    def _calc_h2(self):
        v = self._v
        e = self._e
        return np.mean([linalg.norm(v[i] - v[j])
                        for i, es in enumerate(e)
                        for j in es])**2

    def _calc_D_h2LC_inv(self):
        return splu(self.D.tocsc() - self.h2 * self.LC)

    def _calc_LC_inv(self):
        # Need to add a small offset to guarantee that the inverse exists.
        # Since L_C is negative semidefinite, subracting off a small positive
        # multiple of the identity guarantees that the resulting matrix is
        # invertible. For generality's sake, we pick the magnitude relative to
        # the largest eigenvalue of L_C.
        offset_magnitude = eigsh(self.LC, k=1,
                                 return_eigenvectors=False)[0] * 1e-10
        return splu(self.LC - sparse.eye(self._V) * offset_magnitude)

    def _calc_u(self):
        delta = np.zeros((self._V, 1))
        delta[self._gamma] = 1
        return self.D_h2LC_inv.solve(delta)

    def _calc_q(self):
        v = self._v
        e = self._e
        c = self._c
        u = self.u
        return {(i, j): u[i] * (v[c[i,j]] - v[j])
                for i, es in enumerate(e)
                for j in es}

    def _calc_m(self):
        e = self._e
        c = self._c
        q = self.q
        return {(i, j): q[i,j] + q[j,c[i,j]] + q[c[i,j],i]
                for i, es in enumerate(e)
                for j in es}

    def _calc_grad_u(self):
        e = self._e
        N = self.N
        m = self.m
        return {(i, j): np.cross(N[i,j], m[i,j])
                for i, es in enumerate(e)
                for j in es}

    def _calc_X(self):
        return {(i, j): -grad_u / linalg.norm(grad_u)
                for (i, j), grad_u in self.grad_u.items()}

    def _calc_p(self):
        v = self._v
        e = self._e
        cot = self.cot
        return {(i, j): cot[i,j] * (v[j] -v[i])
                for i, es in enumerate(e)
                for j in es}

    def _calc_div_X(self):
        e = self._e
        c = self._c
        X = self.X
        p = self.p
        return np.array([sum([(p[i,j] - p[c[i,j],i]) @ X[i,j]
                              for j in es]) / 2
                         for i, es in enumerate(e)])

    def calc_phi(self, gamma):
        self._laplacian_forward.calc_L()
        self.N = self._laplacian_forward.N
        self.A = self._laplacian_forward.A
        self.D = self._laplacian_forward.D
        self.cot = self._laplacian_forward.cot
        self.LC = self._laplacian_forward.LC

        updated = False
        if self._updates != self._mesh.updates():
            updated = True
            self._updates = self._mesh.updates()
            self._v = self._mesh.get_vertices()

            self.h2 = self._calc_h2()
            self.D_h2LC_inv = self._calc_D_h2LC_inv()
            self.LC_inv = self._calc_LC_inv()

        if (updated or self._updates != self._mesh.updates()
            or self._gamma != gamma):
            if not updated:
                self._updates = self._mesh.updates()
                self._v = self._mesh.get_vertices()
            self._gamma = gamma

            self.u = self._calc_u()
            self.q = self._calc_q()
            self.m = self._calc_m()
            self.grad_u = self._calc_grad_u()
            self.X = self._calc_X()
            self.p = self._calc_p()
            self.div_X = self._calc_div_X()
            phi = self.LC_inv.solve(self.div_X)
            # We subtract off the minimum here so that we satisfy the obvious
            # initial conidition (namely, the distance from gamma to itself
            # should be 0)
            self.phi = phi - min(phi)

        return self.phi


In [7]:
frequency = 5
M = SphereMesh(frequency)

laplacian_forward = LaplacianForward(M)
geodesic_forward = GeodesicForward(M, laplacian_forward)

phi = geodesic_forward.calc_phi(M.nearest_direction_index(
    SphereMesh.latitude_longitude_to_direction(0, 0)))

rng = np.random.default_rng()
for _ in range(10):
    direction = np.array([rng.random(), rng.random(), rng.random()])
    direction = direction / linalg.norm(direction)
    print(f'Estimated: {phi[M.nearest_direction_index(direction)]:.6f};',
          end=' ')
    print(f'True: {np.arccos(direction[0]):.6f}')


Estimated: 0.418103; True: 0.368619
Estimated: 0.861264; True: 0.905454
Estimated: 0.978127; True: 1.108748
Estimated: 1.424091; True: 1.539715
Estimated: 0.978127; True: 0.990425
Estimated: 1.221053; True: 1.473618
Estimated: 0.717941; True: 0.930585
Estimated: 1.360817; True: 1.458944
Estimated: 1.111680; True: 1.249717
Estimated: 1.422892; True: 1.378131


## Geodesic Gradient Computation

Note that $c(i, c(j, i)) = j$. This is helpful for reindexing some sums (in particular, the one for $\nabla \cdot X$).

We then have the following partial derivatives: $$\begin{aligned}
    \frac{\partial u^\gamma}{\partial \rho_\ell} &= -(D - tL_C)^{-1}\left(\frac{\partial D}{\partial \rho_\ell} - t\frac{\partial L_C}{\partial \rho_\ell}\right)u^\gamma, \\
    \frac{\partial q_{i, j}}{\partial \rho_\ell} &= \begin{cases}
        \displaystyle\frac{\partial u^\gamma_i}{\rho_\ell}(v_{c(i, j)} - v_j) - u^\gamma_i\frac{\partial v_\ell}{\rho_\ell} & \text{if $\ell = j$}, \\
        \displaystyle\frac{\partial u^\gamma_i}{\rho_\ell}(v_{c(i, j)} - v_j) + u^\gamma_i\frac{\partial v_\ell}{\partial \rho_\ell} & \text{if $\ell = c(i, j)$}, \\
        \displaystyle\frac{\partial u^\gamma_i}{\rho_\ell}(v_{c(i, j)} - v_j) & \text{otherwise},
    \end{cases} \\
    \frac{\partial m_{i, j}}{\partial \rho_\ell} &= \frac{\partial q_{i, j}}{\partial \rho_\ell} + \frac{\partial q_{j, c(i, j)}}{\partial \rho_\ell} + \frac{\partial q_{c(i, j), i}}{\partial \rho_\ell}, \\
    \frac{\partial (\widetilde{\nabla} u^\gamma)_{i, j}}{\partial \rho_\ell} &= \frac{\partial N_{i, j}}{\partial \rho_\ell} \times m_{i, j} + N_{i, j} \times \frac{\partial m_{i, j}}{\partial \rho_\ell}, \\
    \frac{\partial X^\gamma_{i, j}}{\partial \rho_\ell} &= -\frac{1}{\|(\widetilde{\nabla} u^\gamma)_{i, j}\|_2}(I - X^\gamma_{i, j}(X^\gamma_{i, j})^\intercal)\frac{\partial (\widetilde{\nabla} u^\gamma)_{i, j}}{\partial \rho_\ell}, \\
    \frac{\partial p_{i, j}}{\partial \rho} &= \begin{cases}
        \displaystyle\left(\frac{\partial}{\partial \rho_\ell}\cot(\theta_{i, j})\right)(v_j - v_i) - \cot(\theta_{i, j})\frac{\partial v_\ell}{\rho_\ell} & \text{if $\ell = i$}, \\
        \displaystyle\left(\frac{\partial}{\partial \rho_\ell}\cot(\theta_{i, j})\right)(v_j - v_i) + \cot(\theta_{i, j})\frac{\partial v_\ell}{\rho_\ell} & \text{if $\ell = j$}, \\
        \displaystyle\left(\frac{\partial}{\partial \rho_\ell}\cot(\theta_{i, j})\right)(v_j - v_i) & \text{if $\ell = c(i, j)$}, \\
        0 & \text{otherwise},
    \end{cases} \\
    \frac{\partial (\nabla \cdot X^\gamma)_i}{\partial \rho_\ell} &= \frac{1}{2}\sum_{\substack{k \\ \text{$(v_i, v_k)$ is an edge}}}\left(\left(\frac{\partial p_{i, k}}{\partial \rho_\ell} - \frac{\partial p_{c(i, k), i}}{\partial \rho_\ell}\right) \cdot X^\gamma_{i, k} + (p_{i, k} - p_{c(i, k), i}) \cdot \frac{\partial X^\gamma_{i, k}}{\partial \rho_\ell}\right) \\
    \frac{\partial \phi^\gamma}{\partial \rho_\ell} &= L_C^+\left(\frac{\partial (\nabla \cdot X^\gamma)}{\partial \rho_\ell} - \frac{\partial L_C}{\partial \rho_\ell}\phi^\gamma\right).
\end{aligned}$$

Some of these can probably be simplified using, for example, the triple product or similar? Also, the pseudoinverse computation might be rather inefficient.

In [8]:
class GeodesicReverse:
    '''
    Implementation of the gradient of the heat method for geodesic distance
    computation. Objects of this class must be initialized with the indices of
    the vertics with respect to which we are differentiating (`l`) as well as
    the partial derivatives of v with repect to rho_l (stored in the same shape
    as `mesh.get_vertices()`).
    '''

    def __init__(self, mesh, geodesic_forward=None, laplacian_reverse=None):
        self._mesh = mesh
        self._updates = self._mesh.updates() - 1
        self._v = None
        self._e = self._mesh.get_edges()
        self._c = self._mesh.get_c()

        self._V = len(self._e)

        self._dif_v = None
        self._ls = None

        self._geodesic_forward = geodesic_forward
        if self._geodesic_forward is None:
            self._geodesic_forward = GeodesicForward(mesh)

        self._laplacian_reverse = laplacian_reverse
        if self._laplacian_reverse is None:
            self._laplacian_reverse = LaplacianReverse(mesh)

        self._N = None
        self._A = None
        self._D = None
        self._cot = None
        self._LC = None
        self._h2 = None
        self._D_h2LC_inv = None
        self._LC_inv = None

        self._dif_v = None

        self._dif_N = None
        self._dif_A = None
        self._dif_D = None
        self._dif_cot = None
        self._dif_LC = None

        self._gamma = None
        self._u = None
        self._q = None
        self._m = None
        self._grad_u = None
        self._X = None
        self._p = None
        self._div_X = None
        self._phi = None

        # Derivatives are stored as maps sending l to the partial with respect
        # to rho_l. The types of the outputs of the maps match the types of
        # what are being differentiated.
        self.dif_u = None
        self.dif_grad_u = None
        self.dif_X = None
        self.dif_p = None
        self.dif_div_X = None
        self.dif_phi = None

    def _calc_dif_u(self, l):
        return -self._D_h2LC_inv.solve((self._dif_D[l].tocsc()
                                        - self._h2 * self._dif_LC[l]) @ self._u)

    def _calc_dif_q(self, l):
        dif_q = {}
        v = self._v
        e = self._e
        c = self._c
        dif_v = self._dif_v[l]
        u = self._u
        dif_u = self.dif_u
        for i, es in enumerate(e):
            for j in es:
                k = c[i,j]
                dif_q[i,j] = dif_u[l][i] * (v[k] - v[j])
                if l == j:
                    dif_q[i,j] -= u[i] * dif_v
                elif l == k:
                    dif_q[i,j] += u[i] * dif_v
        return dif_q

    def _calc_dif_m(self, l):
        e = self._e
        c = self._c
        dif_q = self.dif_q[l]
        return {(i, j): dif_q[i,j] + dif_q[j,c[i,j]] + dif_q[c[i,j],i]
                for i, es in enumerate(e)
                for j in es}

    def _calc_dif_grad_u(self, l):
        dif_grad_u = {}
        e = self._e
        N = self._N
        m = self._m
        dif_N = self._dif_N[l]
        dif_m = self.dif_m[l]
        for i, es in enumerate(e):
            for j in es:
                dif_grad_u[i,j] = np.cross(N[i,j], dif_m[i,j])
                if (i, j) in dif_N:
                    dif_grad_u[i,j] += np.cross(dif_N[i,j], m[i,j])
        return dif_grad_u

    def _calc_dif_X(self, l):
        e = self._e
        grad_u = self._grad_u
        X = self._X
        dif_grad_u = self.dif_grad_u[l]
        return {(i, j): ((X[i,j] @ dif_grad_u[i,j]) * X[i,j] - dif_grad_u[i,j])
                        / linalg.norm(grad_u[i,j])
                for i, es in enumerate(e)
                for j in es}

    def _calc_dif_p(self, l):
        dif_p = {}
        v = self._v
        c = self._c
        dif_v = self._dif_v[l]
        cot = self._cot
        dif_cot = self._dif_cot[l]
        for i, es in enumerate(self._e):
            for j in es:
                if l == i:
                    dif_p[i,j] = dif_cot[i,j] * (v[j] - v[i]) \
                        - cot[i,j] * dif_v
                elif l == j:
                    dif_p[i,j] = dif_cot[i,j] * (v[j] - v[i]) \
                        + cot[i,j] * dif_v
                elif l == c[i,j]:
                    dif_p[i,j] = dif_cot[i,j] * (v[j] - v[i])
        return dif_p

    def _calc_dif_div_X(self, l):
        dif_div_X = np.zeros(self._V)
        e = self._e
        c = self._c
        X = self._X
        p = self._p
        dif_X = self.dif_X[l]
        dif_p = self.dif_p[l]
        for i, es in enumerate(e):
            for j in es:
                k = c[i,j]
                dpij = dif_p[i,j] if (i, j) in dif_p else np.zeros(3)
                dpki = dif_p[k,i] if (k, i) in dif_p else np.zeros(3)
                dif_div_X[i] += ((dpij - dpki) @ X[i,j]
                                 + (p[i,j] - p[k,i]) @ dif_X[i,j]) / 2
        return dif_div_X

    def calc_dif_phi(self, gamma, dif_v, ls=None):
        if self._ls is None:
            self._ls = range(self._V)
        if ls is None:
            ls = range(self._V)

        self._geodesic_forward.calc_phi(gamma)
        self._N = self._geodesic_forward.N
        self._A = self._geodesic_forward.A
        self._D = self._geodesic_forward.D
        self._cot = self._geodesic_forward.cot
        self._LC = self._geodesic_forward.LC
        self._h2 = self._geodesic_forward.h2
        self._D_h2LC_inv = self._geodesic_forward.D_h2LC_inv
        self._LC_inv = self._geodesic_forward.LC_inv
        self._u = self._geodesic_forward.u
        self._q = self._geodesic_forward.q
        self._m = self._geodesic_forward.m
        self._grad_u = self._geodesic_forward.grad_u
        self._X = self._geodesic_forward.X
        self._p = self._geodesic_forward.p
        self._div_X = self._geodesic_forward.div_X
        self._phi = self._geodesic_forward.phi

        self._laplacian_reverse.calc_dif_L(dif_v, ls)
        self._dif_N = self._laplacian_reverse.dif_N
        self._dif_A = self._laplacian_reverse.dif_A
        self._dif_D = self._laplacian_reverse.dif_D
        self._dif_cot = self._laplacian_reverse.dif_cot
        self._dif_LC = self._laplacian_reverse.dif_LC

        if (self._updates != self._mesh.updates() or self._gamma != gamma
            or self._ls != ls):
            self._updates = self._mesh.updates()
            self._v = self._mesh.get_vertices()
            self._gamma = gamma
            self._dif_v = dif_v
            self._ls = ls

            self.dif_u = {l: self._calc_dif_u(l) for l in self._ls}
            self.dif_q = {l: self._calc_dif_q(l) for l in self._ls}
            self.dif_m = {l: self._calc_dif_m(l) for l in self._ls}
            self.dif_grad_u = {l: self._calc_dif_grad_u(l) for l in self._ls}
            self.dif_X = {l: self._calc_dif_X(l) for l in self._ls}
            self.dif_p = {l: self._calc_dif_p(l) for l in self._ls}
            self.dif_div_X = {l: self._calc_dif_div_X(l) for l in self._ls}
            self.dif_phi = {l: self._LC_inv.solve(self.dif_div_X[l]
                                                  - self._LC @ self._phi)
                            for l in self._ls}

        return self.dif_phi


# Objective/Loss Functions

To enforce that the mesh approximates our desired surface, we define the objective functions $$\begin{aligned}
    \mathcal{L}_{\mathrm{geodesic}}(M) &\triangleq \sum_{e \in E_G} (\text{least squares residual of edge \(e\)})^2, \\
    \mathcal{L}_{\mathrm{smooth}}(M) &\triangleq -\rho^\intercal L_C\rho, \\
    \mathcal{L}(M) &\triangleq \mathcal{L}_{\mathrm{geodesic}}(M) + \lambda\mathcal{L}_{\mathrm{smooth}}(M),
\end{aligned}$$ where $\lambda$ is a tunable hyperparameter. Our goal is then to minimize $\mathcal{L}(M)$.

Note that the loss functions (particularly the geodesic and total ones) also have a dependence on the measured latencies. We omit that as a written parameter because they are treated as fixed (we are really optimizing over the manifold, not over the "book" measurements).

# Geodesic Distance Loss Function and Its Gradient Computation

Recall the one-dimensional least squares setup. Here, we wish to relate $t_e$ to $d_e$ by $t_e \approx \beta_0 + \beta_1d_e$, where $d$ is some function of $\phi$. The goal is to minimize $$\sum_{e \in E_G}(t_e - \beta_0 - \beta_1d_e)^2$$ with respect to $\beta_0$ and $\beta_1$. Differentiating with respect to $\beta_0$ and $\beta_1$ gives the following relations: $$\begin{aligned}
    0 &= -2\sum_{e \in E_G}(t_e - \beta_0 - \beta_1d_e), \\
    0 &= -2\sum_{e \in E_G}d_e(t_e - \beta_0 - \beta_1d_e).
\end{aligned}$$

Rearranging, $$\begin{aligned}
    \beta_1\sum_{e \in E_G}d_e &= \sum_{e \in E_G}t_e - \beta_0|E_G|, \\
    \beta_1\sum_{e \in E_G}d_e^2 &= \sum_{e \in E_G}d_et_e - \beta_0\sum_{e \in E_G}d_e.
\end{aligned}$$ Scaling and equating, we have $$\begin{aligned}
    \left(\sum_{e \in E_G}d_e^2\right)\left(\sum_{e \in E_G}t_e\right) - \beta_0|E_G|\left(\sum_{e \in E_G}d_e^2\right) &= \left(\sum_{e \in E_G}d_e\right)\left(\sum_{e \in E_G}d_et_e\right) - \beta_0\left(\sum_{e \in E_G}d_e\right)^2 \\
    \beta_0 &= \frac{\left(\sum_{e \in E_G}d_e^2\right)\left(\sum_{e \in E_G}t_e\right) - \left(\sum_{e \in E_G}d_e\right)\left(\sum_{e \in E_G}d_et_e\right)}{|E_G|\left(\sum_{e \in E_G}d_e^2\right) - \left(\sum_{e \in E_G}d_e\right)^2}.
\end{aligned}$$

We similarly find $$\begin{aligned}
    \beta_0|E_G| &= \sum_{e \in E_G}t_e - \beta_1\sum_{e \in E_G}d_e, \\
    \beta_0\sum_{e \in E_G}d_e &= \sum_{e \in E_G}d_et_e - \beta_1\sum_{e \in E_G}d_e^2,
\end{aligned}$$ so $$\begin{aligned}
    \left(\sum_{e \in E_G}d_e\right)\left(\sum_{e \in E_G}t_e\right) - \beta_1\left(\sum_{e \in E_G}d_e\right)^2 &= |E_G|\sum_{e \in E_G}d_et_e - \beta_1|E_G|\sum_{e \in E_G}d_e^2 \\
    \beta_1 &= \frac{|E_G|\sum_{e \in E_G}d_et_e - \left(\sum_{e \in E_G}d_e\right)\left(\sum_{e \in E_G}t_e\right)}{|E_G|\left(\sum_{e \in E_G}d_e^2\right) - \left(\sum_{e \in E_G}d_e\right)^2}.
\end{aligned}$$

These expressions are quite bad, but we can simplify them somewhat if we make the following assumptions: $$\begin{aligned}
    1 &= \frac{1}{|E_G|}\sum_{e \in E_G}d_e^2 , \\
    0 &= \sum_{e \in E_G}d_e.
\end{aligned}$$ Then $$\begin{aligned}
    \beta_0 &= \frac{1}{|E_G|}\sum_{e \in E_G}t_e, \\
    \beta_1 &= \frac{1}{|E_G|}\sum_{e \in E_G}d_et_e.
\end{aligned}$$ The assumptions hold if we have $$\begin{aligned}
    \widetilde{d}_e &\triangleq \phi_e - \frac{1}{|E_G|}\sum_{e' \in E_G}\phi_{e'}, \\
    d_e &\triangleq \frac{\widetilde{d}_e}{\sqrt{\frac{1}{|E_G|}\sum_{e' \in E_G}\widetilde{d}_{e'}^2}}.
\end{aligned}$$ Importantly, these are just affine transformations.

Then $$\mathcal{L}_{\mathrm{geodesic}}(M) \triangleq \sum_{e \in E_G}\left(t_e - \frac{1}{|E_G|}\sum_{e' \in E_G}t_{e'} - \frac{d_e}{|E_G|}\sum_{e' \in E_G}d_{e'}t_{e'}\right)^2$$

In [9]:
class LinearRegressionForward:
    def __init__(self):
        self._phi = None
        self._t = None

        self.d_tilde = None
        self.d = None
        self.residuals = None
        self.lse = None

        self.beta = None

    def calc_lse(self, phi, t):
        E = phi.shape[0]

        # If phi and t haven't changed, don't do any additional work.
        if (self._phi is not None and self._phi.shape == phi.shape
            and np.allclose(self._phi, phi)
            and self._t is not None and self._t.shape == t.shape
            and np.allclose(self._t, t)
            and self.lse is not None):
            return self.lse

        self._phi = np.copy(phi)
        self._t = np.copy(t)

        self.d_tilde = phi - np.sum(phi) / E
        self.d = self.d_tilde / (self.d_tilde @ self.d_tilde / E)**0.5
        self.residuals = t - np.sum(t) / E - (self.d @ t / E) * self.d
        self.lse = self.residuals @ self.residuals
        return self.lse

    def calc_beta(self, phi, t):
        E = phi.shape[0]

        # If phi and t haven't changed, don't do any additional work.
        if (self._phi is not None and self._phi.shape == phi.shape
            and np.allclose(self._phi, phi)
            and self._t is not None and self._t.shape == t.shape
            and np.allclose(self._t, t)
            and self.beta is not None):
            return self.beta

        # Note that the following disagrees with the notation in the writeup
        # above. In particular, beta here is the coefficients when relating phi
        # to t (as opposed to relating d to t).
        denominator = E * (phi @ phi) - np.sum(phi)**2
        self.beta = (
            (phi @ phi * np.sum(t) - np.sum(phi) * (phi @ t)) / denominator,
            (E * (phi @ t) - np.sum(phi) * np.sum(t)) / denominator,
        )
        return self.beta


Furthermore, $$\begin{aligned}
    \frac{\partial}{\partial \rho_\ell}\mathcal{L}_{\mathrm{geodesic}}(M) &= -\frac{2}{|E_G|}\sum_{e \in E_G}\left(t_e - \frac{1}{|E_G|}\sum_{e' \in E_G}t_{e'} - \frac{d_e}{|E_G|}\sum_{e' \in E_G}d_{e'}t_{e'}\right) \\
        &\hspace{8em}\cdot \left(\frac{\partial d_e}{\partial \rho_\ell}\sum_{e' \in E_G}d_{e'}t_{e'} + d_e\sum_{e' \in E_G}\frac{\partial d_{e'}}{\partial \rho_\ell}t_{e'}\right), \\
    \frac{\partial d_e}{\partial \rho_\ell} &= \frac{\sqrt{\sum_{e' \in E_G}\widetilde{d}_{e'}^2} \cdot \frac{\partial \widetilde{d}_e}{\partial \rho_\ell} - d_e\sum_{e' \in E_G}\left(\widetilde{d}_{e'} \cdot \frac{\partial \widetilde{d}_{e'}}{\partial \rho_\ell}\right)}{\sum_{e' \in E_G}\widetilde{d}_{e'}^2}, \\
        &= \frac{1}{\sqrt{\sum_{e' \in E_G}\widetilde{d}_{e'}^2}}\left(\frac{\partial \widetilde{d}_e}{\partial \rho_\ell} - d_e\sum_{e' \in E_G}\left(d_{e'} \cdot \frac{\partial \widetilde{d}_{e'}}{\partial \rho_\ell}\right)\right) \\
    \frac{\partial \widetilde{d}_e}{\partial \rho_\ell} &= \frac{\partial \phi_e}{\partial \rho_\ell} - \frac{1}{|E_G|}\sum_{e' \in E_G}\frac{\partial \phi_{e'}}{\partial \rho_\ell}.
\end{aligned}$$

In [10]:
class LinearRegressionReverse:
    def __init__(self, linear_regression_forward=None):
        self._phi = None
        self._t = None
        self._dif_phi = None

        self._ls = None

        self._linear_regression_forward = linear_regression_forward
        if linear_regression_forward is None:
            self._linear_regression_forward = LinearRegressionForward()

        self._d_tilde = None
        self._d = None
        self._residuals = None
        self._lse = None

        self.dif_d_tilde = None
        self.dif_d = None
        self.dif_residuals = None
        self.dif_lse = None

    def calc_dif_lse(self, phi, t, dif_phi, ls=None):
        E = phi.shape[0]
        if self._ls is None:
            self._ls = range(E)
        if ls is None:
            ls = range(E)

        pair_changed = False
        if (self._phi is None or self._phi.shape != phi.shape
            or not np.allclose(self._phi, phi)
            or self._t is None or self._t.shape != t.shape
            or not np.allclose(self._t, t) or list(self._ls) != list(ls)
            or self._dif_phi is None
            or not np.all([l in self._dif_phi for l in self._ls])
            or not np.all([l in dif_phi for l in self._ls])
            or not np.all([np.allclose(self._dif_phi[l], dif_phi[l])
                           for l in self._ls])):
            pair_changed = True
            self._phi = np.copy(phi)
            self._t = np.copy(t)
            self._dif_phi = dif_phi

            self._ls = ls

            self._linear_regression_forward.calc_lse(self._phi, self._t)
            self._d_tilde = self._linear_regression_forward.d_tilde
            self._d = self._linear_regression_forward.d
            self._residuals = self._linear_regression_forward.residuals
            self._lse = self._linear_regression_forward.lse

        if not pair_changed:
            return self.dif_lse

        self.dif_d_tilde = {l: self._dif_phi[l] - np.sum(self._dif_phi[l]) / E for l in self._ls}
        self.dif_d = {l: (self.dif_d_tilde[l] - (self._d @ self.dif_d_tilde[l]) * self._d) / linalg.norm(self._d_tilde) for l in self._ls}
        self.dif_residuals = {l: (-self._d @ self._t * self.dif_d[l] - self.dif_d[l] @ self._t * self._d) / E for l in self._ls}
        self.dif_lse = {l: 2 * self._residuals @ self.dif_residuals[l] for l in self._ls}
        return self.dif_lse


# Smoothness Loss Function and Its Gradient Computation

It would be nice to have something like [SI-MVS](https://www2.eecs.berkeley.edu/Pubs/TechRpts/1993/CSD-93-732.pdf#page=142) (see [this](https://www2.eecs.berkeley.edu/Pubs/TechRpts/1992/CSD-92-664.pdf) too). [Willmore Energy](https://en.wikipedia.org/wiki/Willmore_energy) is another possibility.

[This](https://www.cad-journal.net/files/vol_4/CAD_4(5)_2007_607-617.pdf) gives a good overview.

For now, we use $$\mathcal{L}_{\text{smooth}}(M) \triangleq -\rho^\intercal L_C\rho.$$

In [11]:
class SmoothForward:
    def __init__(self, mesh, laplacian_forward=None):
        self._mesh = mesh
        self._rho = None

        self._updates = self._mesh.updates() - 1

        self._laplacian_forward = laplacian_forward
        if self._laplacian_forward is None:
            self._laplacian_forward = LaplacianForward(mesh)

        self.LC = None
        self.L_smooth = None

    def calc_L_smooth(self):
        self._laplacian_forward.calc_L()
        self.LC = self._laplacian_forward.LC

        if self._updates != self._mesh.updates():
            self._updates = self._mesh.updates()
            self._rho = self._mesh.get_rho()

            self.L_smooth = -self._rho.T @ (self.LC @ self._rho)

        return self.L_smooth


We compute the derivatives $$\begin{aligned}
    \frac{\partial}{\partial \rho_\ell}\mathcal{L}_{\text{smooth}}(M) &= -e_\ell^\intercal L_C\rho - \rho^\intercal\frac{\partial L_C}{\partial \rho_\ell}\rho - \rho^\intercal L_Ce_\ell.
\end{aligned}$$

In [12]:
class SmoothReverse:
    def __init__(self, mesh, laplacian_forward=None, laplacian_reverse=None):
        self._mesh = mesh
        self._updates = self._mesh.updates() - 1
        self._rho = None

        self._dif_v = None
        self._ls = None

        self._laplacian_forward = laplacian_forward
        if self._laplacian_forward is None:
            self._laplacian_forward = LaplacianForward(mesh, )

        self._laplacian_reverse = laplacian_reverse
        if self._laplacian_reverse is None:
            self._laplacian_reverse = LaplacianReverse(mesh, laplacian_forward)

        self._LC = None
        self.dif_LC = None

        self.dif_L_smooth = None

    def calc_dif_L_smooth(self, dif_v, ls=None):
        if self._ls is None:
            self._ls = dif_v.keys()
        if ls is None:
            ls = dif_v.keys()

        self._laplacian_forward.calc_L()
        self._LC = self._laplacian_forward.LC

        self._laplacian_reverse.calc_dif_L(dif_v, ls)
        self.dif_LC = self._laplacian_reverse.dif_LC

        if self._updates != self._mesh.updates() or self._ls != ls:
            self._updates = self._mesh.updates()
            self._rho = self._mesh.get_rho()
            self._dif_v = dif_v
            self._ls = ls

            L_rho = self._LC @ self._rho + self._LC.T @ self._rho
            self.dif_L_smooth = {l: -L_rho[l]
                                    - self._rho @ (self.dif_LC[l] @ self._rho)
                                 for l in self._ls}

        return self.dif_L_smooth


# Putting It All Together with Minibatch Gradient Descent

We can rewrite $$\mathcal{L}_{\text{geodesic}}(M) = \sum_{v \in V_G} \sum_{\substack{v' \in V_G \\ (v, v') \in E_G}} \frac{1}{2}(\text{least squares residual of edge \((v, v')\)})^2.$$ From here, we can take the standard approach of batching the gradient computation by vertices. This idea fits well with the heat method, since the heat method necessarily computes all geodesic distances (and their gradients) from a single source (represented by $v$ in the above decomposition).

Motivated by this, define $$\mathcal{L}_{\text{geodesic}, v}(M) = \sum_{\substack{v' \in V_G \\ (v, v') \in E_G}} \frac{1}{2}(\text{least squares residual of edge \((v, v')\)})^2.$$

## Partial Derivative Selection

Computing the gradient of $\mathcal{L}_{\text{geodesic}, v}(M)$ is quite expensive. We can make the computation more efficient by approximately reconstructing the geodesics from $v$. Importantly, for vertices not on the faces through which the geodesics pass, the partial derivatives will be $0$.

### Fixed Point Iteration Strategy

Consider the set of geodesics between $v_a$ and $v_b$ for $b \in B$, where the geodesic distance is computed with $\gamma = \{v_a\}$. A conservative estimate of these faces incident to these geodesics can be constructed iteratively by finding a minimal fixed point $S_{a, B}$ of $$S \mapsto \left\{(v_i \to v_j \to v_{c(i, j)}) : \begin{array}{c}\text{$(v_i \to v_j \to v_{c(i, j)})$ is adjacent to a face $(v_j \to v_i \to v_{c(j, i)}) \in S$} \\ \text{and $\phi^\gamma_{c(i, j)} < \max(\phi^\gamma_i, \phi^\gamma_j)$}\end{array}\right\}$$ such that $$\{(v_b \to v_i \to v_{c(b, i)}) : \phi^\gamma_b > \min(\phi^\gamma_i, \phi^\gamma_{c(b, i)}), b \in B\} \subseteq S_{a, B}.$$ In other words, $S_{a, B}$ is (at least) the set of faces through which paths that always move towards $v_a$ can pass through.

With this computed, the vertices we care about are those incident to the faces in $S_{a, B}$.

In [13]:
def approximate_geodesics_fpi(mesh, phi, initial_vertices):
    e = mesh.get_edges()
    c = mesh.get_c()
    vertices = set()
    to_process = []
    processed = set()
    for b in initial_vertices:
        for i in e[b]:
            cbi = c[b,i]
            if phi[b] > phi[i] or phi[b] > phi[cbi]:
                to_process.append((b, i, cbi))

    while to_process:
        (i, j, k) = to_process[-1]
        del to_process[-1]
        if j < i and j < k:
            j, k, i = i, j, k
        elif k < i and k < j:
            k, i, j = i, j, k
        if (i, j, k) in processed:
            continue

        vertices.add(i)
        vertices.add(j)
        vertices.add(k)

        cji = c[j,i]
        if phi[cji] < phi[i] or phi[cji] < phi[j]:
            to_process.append((j, i, cji))

        ckj = c[k,j]
        if phi[ckj] < phi[j] or phi[ckj] < phi[k]:
            to_process.append((k, j, ckj))

        cik = c[i,k]
        if phi[cik] < phi[k] or phi[cik] < phi[i]:
            to_process.append((i, k, cik))

        processed.add((i, j, k))

    return vertices


### Triangle Inequality Strategy

Suppose we have the same setup as before, where we want to find the faces incident to the geodesic between $v_a$ and $v_b$. Let's focus in on a face $f = (v_i \to v_j \to v_{c(i, j)})$. Note that geodesic distance satisfies the triangle inequality. Define $$\begin{aligned}
    \widetilde{a} \in \argmin_{\ell \in \{i, j, c(i, j)\}}\left(\phi^{\{v_a\}}_\ell\right), \\
    \widetilde{b} \in \argmin_{\ell \in \{i, j, c(i, j)\}}\left(\phi^{\{v_b\}}_\ell\right).
\end{aligned}$$ From the triangle inequality, we have $$\phi^{\{v_a\}}_b \le \phi^{\{v_a\}}_{\widetilde{a}} + \|v_{\widetilde{a}} - v_{\widetilde{b}}\|_2 + \phi^{\{v_b\}}_{\widetilde{b}}.$$ Intuitively, this inequality is saying that $f$ cannot be too far away from $v_a$ and $v_b$.

Note that any $f$ that satisfies the above also automatically satisfies the conditions of the fixed point strategy (the "monotonic" path is the concatenation of the geodesic from $v_b$ to $v_{\widetilde{b}}$, the edge from $v_{\widetilde{b}}$ to $v_{\widetilde{a}}$ (or no edge if $\widetilde{b} = \widetilde{a}$), and the geodesic from $v_{\widetilde{a}}$ to $v_a$). Therefore, this strategy is a refinement of the previous one. Unfortunately, the previous strategy required thinking only of $\phi^{\{v_a\}}$, whereas this strategy requires considering additionally $\phi^{\{v_b\}}$ for each $b$ such that $(a, b) \in E_G$.

In [14]:
# TODO: Implement the triangle inequality strategy


In [15]:
# Construct the mesh
frequency = 2
M = SphereMesh(frequency)
rng = np.random.default_rng()
directions = M.get_directions()
V = directions.shape[0]
dif_v = {l: directions[l] for l in range(V)}
rho = rng.random(V) + 0.5
rho /= sum(linalg.norm(rho[l]) for l in range(V)) / V
M.set_rho(rho)


In [16]:
# Get some (phony) latency measurements
lat_long_pairs = [
    (0, 0),
    (0, 90),
    (90, 0),
    (0, 180),
    (0, -90),
    (-90, 0),
]
directions = [SphereMesh.latitude_longitude_to_direction(lat, long) for (lat, long) in lat_long_pairs]
s_indices = [M.nearest_direction_index(direction) for direction in directions]
ts = {si: [(sj, np.arccos(dsi @ dsj))
           for j, (sj, dsj) in enumerate(zip(s_indices, directions)) if (i - j) % 6 != 0]
      for i, (si, dsi) in enumerate(zip(s_indices, directions))}


In [17]:
# Construct the differentiation heirarchy
laplacian_forward = LaplacianForward(M)
geodesic_forward = GeodesicForward(M, laplacian_forward)
linear_regression_forward = LinearRegressionForward()
smooth_forward = SmoothForward(M)
laplacian_reverse = LaplacianReverse(M, laplacian_forward)
geodesic_reverse = GeodesicReverse(M, geodesic_forward, laplacian_reverse)
linear_regression_reverse = LinearRegressionReverse(linear_regression_forward)
smooth_reverse = SmoothReverse(M)


In [18]:
# Run gradient descent

lam = 0.1

def get_losses(s_indices, ts):
    lse = 0
    for s_index in np.random.permutation(s_indices):
        gamma = [s_index]
        s_connected, t = zip(*ts[s_index])
        s_connected = list(s_connected)
        t = np.array(t)
        phi = geodesic_forward.calc_phi(gamma)
        lse += linear_regression_forward.calc_lse(phi[s_connected], t)
    L_smooth = smooth_forward.calc_L_smooth()
    return lse, L_smooth

rng = np.random.default_rng()

animation_3D = Animation3D()

max_iterations = 2
for i in itertools.count(1):
    if i > max_iterations:
        break

    eta = 1 / i

    lse, L_smooth = get_losses(s_indices, ts)
    print(f'iteration {i}: \n\tlse: {lse:.6f}\n\tL_smooth: {L_smooth:.6f}\n\tLoss: {(lse + lam * L_smooth):.6f}')

    for s_index in np.random.permutation(s_indices):
        animation_3D.add_frame(M)
        gamma = [s_index]
        s_connected, t = zip(*ts[s_index])
        s_connected = list(s_connected)
        t = np.array(t)

        dif_L = np.zeros(V)
        dif_L_smooth = smooth_reverse.calc_dif_L_smooth(dif_v)
        for l in range(V):
            dif_L[l] +=  lam * dif_L_smooth[l]

        phi = geodesic_forward.calc_phi(gamma)
        ls = approximate_geodesics_fpi(M, phi, s_connected)
        dif_phi = geodesic_reverse.calc_dif_phi(gamma, dif_v, ls)
        dif_phi = {l: dif[s_connected] for l, dif in dif_phi.items()}
        dif_lse = linear_regression_reverse.calc_dif_lse(phi[s_connected], t, dif_phi, ls)

        for l in dif_lse:
            dif_L[l] += dif_lse[l]

        rho = np.maximum(rho - eta * dif_L, 0.01)
        rho /= sum(linalg.norm(rho[l]) for l in range(V)) / V
        M.set_rho(rho)

lse, L_smooth = get_losses(s_indices, ts)
print(f'\nFinal lse: {lse:.6f}\nFinal L_smooth: {L_smooth:.6f}\nFinal Loss: {(lse + lam * L_smooth):.6f}')
animation_3D.add_frame(M)


iteration 1: 
	lse: 0.752538
	L_smooth: 7.773550
	Loss: 1.529893
iteration 2: 
	lse: 0.010437
	L_smooth: 0.024723
	Loss: 0.012910

Final lse: 0.003292
Final L_smooth: 0.005288
Final Loss: 0.003821


In [19]:
animation_3D.get_fig(duration=50).show()

# The following values should hopefully both be near 1.0
print(np.min(rho), np.max(rho))


0.9707884631430196 1.0225346480811677


TODO:
* ~~Descend from a noisy sphere (or other simple surfaces) to a "clean" one~~
* ~~Add visualization of GD~~
* Try nonuniform meshes (e.g., more detail in America, less in the oceans)
* ~~Try running with $\lambda = 0$ (and see what happens with different initializations)~~
* Is it possible to first optimize over geodesics, and then optimize over smoothness? (My gut instinct is no, but maybe it could be possible under certain circumstances)

Next steps:
* Other smoothness functions
* What's a good value for $\eta$? Just looking for something that makes the output "pretty good"
* Same with $\lambda$
* Later step is to plot the graph representing the network on the sphere